## Import Libraries and API Key

In [1]:
import os
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SequentialChain, ConversationChain
from langchain.prompts import PromptTemplate
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from secret_key import openai_key

os.environ['OPENAI_API_KEY'] = openai_key

## Choose the model to use.

In [2]:
# Initialize the LLM with the specific model
llm = OpenAI(temperature=0.5)

c:\Users\Furkan Salman\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


## Define the prompt template.

In [3]:
# Define the prompt template
poem_start_prompt = PromptTemplate(
    input_variables=["word"],
    template="Write a poem that describes the word '{word}' in a simple way, to teach a child. Write only one stanza.",

)

# Define the prompt template for adding a stanza to the poem
poem_complete_prompt = PromptTemplate(
    input_variables=["poem_start"],
    template="Complete the poem, that starts as :\n{poem_start}"
)

## Build the chain.

In [4]:
# Build a chain by specifying the LLM and the prompt template
poem_start_chain = LLMChain(llm=llm, prompt=poem_start_prompt, output_key="poem_start")
poem_complete_chain = LLMChain(llm=llm, prompt=poem_complete_prompt, output_key="poem")

# Build a sequential chain by specifying the chains
chain = SequentialChain(
    chains=[poem_start_chain, poem_complete_chain],
    input_variables=["word"],
    output_variables=["poem"],
    verbose=True  # To see the intermediate steps
)

# Run the chain with a specific word
response = chain({"word": "dog"})

print(response)

c:\Users\Furkan Salman\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\Furkan Salman\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.
{'word': 'dog', 'poem': "\n\nIt'll cuddle up when you're feeling down\nAnd chase away any hint of a frown\nA loyal companion, through thick and thin\nA dog's love knows no end or begin\n\nWith eyes full of love, it'll look up at you\nAnd its heart will always remain pure and true\nA dog is more than just a pet\nIt's a faithful friend, you'll never forget\n\nThrough laughter and tears, it'll stand by your side\nAnd in its presence, all worries will subside\nA dog's love is unconditional and pure\nA bond that will forever endure\n\nSo let's cherish these furry friends we hold dear\nFor their love and loyalty, we should always revere\nA dog is a blessing, a gift from above\nA true friend, forever and always, with love."}


In [5]:
print(response["poem"])



It'll cuddle up when you're feeling down
And chase away any hint of a frown
A loyal companion, through thick and thin
A dog's love knows no end or begin

With eyes full of love, it'll look up at you
And its heart will always remain pure and true
A dog is more than just a pet
It's a faithful friend, you'll never forget

Through laughter and tears, it'll stand by your side
And in its presence, all worries will subside
A dog's love is unconditional and pure
A bond that will forever endure

So let's cherish these furry friends we hold dear
For their love and loyalty, we should always revere
A dog is a blessing, a gift from above
A true friend, forever and always, with love.


## Load Tools to Use

In [6]:
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Initialize the agent
# The agent type is ZERO_SHOT_REACT_DESCRIPTION, this means that the agent will deside to use any tool to generate the output if needed.
agent = initialize_agent(
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    llm=llm,
    tools=tools,
    verbose=True # To see the intermediate steps
)

agent.run("When was the first computer invented? Add 50 to the answer.")



> Entering new AgentExecutor chain...


c:\Users\Furkan Salman\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
c:\Users\Furkan Salman\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 We can use Wikipedia to find the answer to this question.
Action: wikipedia
Action Input: First computer
Observation: Page: History of computing hardware
Summary: The history of computing hardware covers the developments from early simple devices to aid calculation to modern day computers.
The first aids to computation were purely mechanical devices which required the operator to set up the initial values of an elementary arithmetic operation, then manipulate the device to obtain the result. Later, computers represented numbers in a continuous form (e.g. distance along a scale, rotation of a shaft, or a voltage). Numbers could also be represented in the form of digits, automatically manipulated by a mechanism. Although this approach generally required more complex mechanisms, it greatly increased the precision of results. The development of transistor technology and then the integrated circuit chip led to a series of breakthroughs, starting with transistor computers and then integrate

'1960'

## Define Memory

Chains do not have memory by default.

In [7]:
type(chain.memory)

NoneType

In [8]:
# Create a memory object
memory = ConversationBufferMemory()

# Add the memory to the chain
chain = SequentialChain(
    chains=[poem_start_chain, poem_complete_chain],
    input_variables=["word"],
    output_variables=["poem"],
    verbose=True,
    memory=memory
)

poem = chain({"word": "dog"})

print(poem)



> Entering new SequentialChain chain...

> Finished chain.
{'word': 'dog', 'history': '', 'poem': " \n\nHe'll cuddle up close, \nWhen you're feeling down, \nHis warm, furry presence, \nWill erase your frown. \nHe'll listen intently, \nTo all that you say, \nWith eyes full of understanding, \nHe'll never stray. \n\nHe'll protect and defend, \nWith all of his might, \nA fearless protector, \nIn the dark of the night. \nHe'll be by your side, \nThrough thick and thin, \nA faithful companion, \nThrough thick and thin. \n\nA dog is a friend, \nA true and loyal mate, \nWith a heart full of love, \nAnd no room for hate. \nSo cherish your pup, \nFor they're a gift from above, \nA dog is a friend, \nAnd that's a fact we can't deny and can't change, \nThat's why we love them, \nAnd they'll forever be our best friend."}


We now have a memory that keeps track of the prompt-response pairs.

In [9]:
print(chain.memory.buffer)

Human: dog
AI:  

He'll cuddle up close, 
When you're feeling down, 
His warm, furry presence, 
Will erase your frown. 
He'll listen intently, 
To all that you say, 
With eyes full of understanding, 
He'll never stray. 

He'll protect and defend, 
With all of his might, 
A fearless protector, 
In the dark of the night. 
He'll be by your side, 
Through thick and thin, 
A faithful companion, 
Through thick and thin. 

A dog is a friend, 
A true and loyal mate, 
With a heart full of love, 
And no room for hate. 
So cherish your pup, 
For they're a gift from above, 
A dog is a friend, 
And that's a fact we can't deny and can't change, 
That's why we love them, 
And they'll forever be our best friend.


In [10]:
# Create a conversation chain without memory
convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


We are able to see the system prompt that includes the history that ConverSationChain keeps by default.

In [11]:
convo.run("Which country won the 2018 FIFA World Cup?")

" France won the 2018 FIFA World Cup. They defeated Croatia 4-2 in the final match. It was their second World Cup victory, the first being in 1998. The tournament was held in Russia from June 14 to July 15, 2018. France's star player, Kylian Mbappé, became the second teenager in history to score a goal in a World Cup final."

In [12]:
convo.run("Who scored the goals in the final match?")

' In the final match, Antoine Griezmann, Paul Pogba, and Kylian Mbappé scored for France, while Ivan Perišić and Mario Mandžukić scored for Croatia.'

In [13]:
print(convo.memory.buffer)

Human: Which country won the 2018 FIFA World Cup?
AI:  France won the 2018 FIFA World Cup. They defeated Croatia 4-2 in the final match. It was their second World Cup victory, the first being in 1998. The tournament was held in Russia from June 14 to July 15, 2018. France's star player, Kylian Mbappé, became the second teenager in history to score a goal in a World Cup final.
Human: Who scored the goals in the final match?
AI:  In the final match, Antoine Griezmann, Paul Pogba, and Kylian Mbappé scored for France, while Ivan Perišić and Mario Mandžukić scored for Croatia.


Every single time we prompt the model, we will pass the memory to it.\
This might be costly therefore we need to limit the memory size or the number of prompt-response pairs to keep in memory.\
To keep the memory size limited we can use a windowed memory.

In [14]:
# k=1 means that the memory will store the last 1 conversation.
memory = ConversationBufferWindowMemory(k=1)

# Create a chain with the limited memory
chain = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)

chain.run("Which country won the 2018 FIFA World Cup?")

" France won the 2018 FIFA World Cup. They defeated Croatia in the final match with a score of 4-2. This was France's second time winning the World Cup, the first being in 1998. The tournament was held in Russia and lasted from June 14 to July 15, 2018. Did you know that the French team was led by coach Didier Deschamps and star players such as Kylian Mbappé, Antoine Griezmann, and Paul Pogba?"

In [15]:
chain.run("What's 1+1?")

' 1+1 is 2. This is a basic mathematical equation that involves adding two numbers together. Did you know that the concept of addition dates back to ancient civilizations such as the Egyptians and Babylonians? They used different methods to represent and calculate numbers, such as tally marks and clay tokens. It wasn\'t until the 16th century that the modern symbol for addition, "+", was first used by German mathematician Johann Widman.'

In [16]:
chain.run("Who scored the goals in the final match?")

' I do not have enough context to accurately answer that question. Can you provide more information about which match you are referring to?'

Since our memory is limited to the last one prompt-response pair, llm does not have the information about the previous prompt\
we have specified the football match therefore it's having trouble to generate a response.